In [47]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint

In [48]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [49]:
soup_corona = bs(html_corona, 'html.parser')

In [50]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [51]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]

corona_df.head(100)
corona_df.to_csv('new_data.csv')
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
0,China,80753,+18,3136.0,17.0,59898.0,17719,5111.0,56.1
1,Italy,9172,"+1,797",463.0,97.0,724.0,7985,733.0,151.7
2,S. Korea,7478,+165,53.0,3.0,166.0,7259,36.0,145.9
3,Iran,7161,+595,237.0,43.0,2394.0,4530,0.0,85.3
4,France,1412,+203,30.0,11.0,12.0,1370,66.0,21.6
...,...,...,...,...,...,...,...,...,...
108,Liechtenstein,1,0,0.0,0.0,0.0,1,0.0,0.0
109,Moldova,1,0,0.0,0.0,0.0,1,0.0,0.2
110,Paraguay,1,0,0.0,0.0,0.0,1,0.0,0.1
111,St. Barth,1,0,0.0,0.0,0.0,1,0.0,0.0


In [52]:
corona_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 0 to 112
Data columns (total 9 columns):
Country,Other       113 non-null object
TotalCases          113 non-null int64
NewCases            113 non-null object
TotalDeaths         113 non-null float64
NewDeaths           113 non-null float64
TotalRecovered      113 non-null float64
ActiveCases         113 non-null int64
Serious,Critical    113 non-null float64
Tot Cases/1M pop    113 non-null float64
dtypes: float64(5), int64(2), object(2)
memory usage: 8.8+ KB


In [53]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)

#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)



df_country_list = df_used["Country"].tolist()
df_used
#df_country_list

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Longitude,Latitude
0,China,80753,18,3136.0,17.0,59898.0,17719,5111.0,56.1,103.819074,36.561765
1,Italy,9172,1797,463.0,97.0,724.0,7985,733.0,151.7,12.070013,42.796626
2,S. Korea,7478,165,53.0,3.0,166.0,7259,36.0,145.9,127.839161,36.385240
3,Iran,7161,595,237.0,43.0,2394.0,4530,0.0,85.3,54.274070,32.575033
4,France,1412,203,30.0,11.0,12.0,1370,66.0,21.6,-2.761729,42.173440
...,...,...,...,...,...,...,...,...,...,...,...
99,Burkina Faso,1,1,0.0,0.0,0.0,1,0.0,0.0,-1.754566,12.269538
100,Liechtenstein,1,0,0.0,0.0,0.0,1,0.0,0.0,9.535743,47.136658
101,Moldova,1,0,0.0,0.0,0.0,1,0.0,0.2,28.456734,47.194988
102,Paraguay,1,0,0.0,0.0,0.0,1,0.0,0.1,-58.400137,-23.228239


In [43]:
df_used.to_csv('data.csv')

In [44]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [45]:
#test_df = corona_df[:2].copy()
corona_json

'[{"Country":"China","TotalCases":80739,"NewCases":"4","TotalDeaths":3120.0,"NewDeaths":1.0,"TotalRecovered":58749.0,"ActiveCases":18870,"CriticalCases":5111.0,"Tot\\u00a0Cases\\/1M pop":56.1,"Longitude":103.8190735,"Latitude":36.56176546},{"Country":"Italy","TotalCases":9172,"NewCases":"1797","TotalDeaths":463.0,"NewDeaths":97.0,"TotalRecovered":724.0,"ActiveCases":7985,"CriticalCases":733.0,"Tot\\u00a0Cases\\/1M pop":151.7,"Longitude":12.07001339,"Latitude":42.79662641},{"Country":"S. Korea","TotalCases":7478,"NewCases":"165","TotalDeaths":53.0,"NewDeaths":3.0,"TotalRecovered":166.0,"ActiveCases":7259,"CriticalCases":36.0,"Tot\\u00a0Cases\\/1M pop":145.9,"Longitude":127.8391609,"Latitude":36.38523983},{"Country":"Iran","TotalCases":7161,"NewCases":"595","TotalDeaths":237.0,"NewDeaths":43.0,"TotalRecovered":2394.0,"ActiveCases":4530,"CriticalCases":0.0,"Tot\\u00a0Cases\\/1M pop":85.3,"Longitude":54.27407004,"Latitude":32.57503292},{"Country":"France","TotalCases":1412,"NewCases":"203"